# CodeHealer Interactive Tutorial

This notebook walks you through a quickstart experience, a deep dive into the agents that power CodeHealer, and ideas for extending the tooling for your own automation needs.

## Quickstart: Heal a Repository in Minutes

1. Point CodeHealer at the root of a Python repository.
2. Provide an OpenAI API key in your environment (`OPENAI_API_KEY`).
3. Launch the orchestrator to automatically repair dependency and runtime issues.

In [ ]:
from codehealer.core.healer import Healer

# Replace this path with the project you want to heal.
target_repo_path = "../some_python_project"

healer = Healer(repo_path=target_repo_path, max_iterations=5)
healer.heal()

# The orchestrator will:
# 1. Create an isolated sandbox environment.
# 2. Install dependencies and repair requirements issues.
# 3. Run the app's entry point and automatically patch runtime errors.

> **Tip:** The sandbox is created *inside* the container, so your host environment remains untouched.

## Deep Dive: Understanding the Agents

CodeHealer relies on two specialized agents built on top of OpenAI models:

- **EnvironmentAgent**: Repairs `requirements.txt` files when `pip install` fails.
- **CodeAgent**: Analyses Python tracebacks and returns a fully patched source file.

You can inspect and even subclass these agents to add guardrails or telemetry.

In [ ]:
from codehealer.agents.environment_agent import EnvironmentAgent
from codehealer.agents.code_agent import CodeAgent

repo_path = "../some_python_project"
env_agent = EnvironmentAgent(repo_path)
code_agent = CodeAgent(repo_path)

pip_error_log = "Collect the pip stderr here..."
requirements_text = open(f"{repo_path}/requirements.txt", encoding="utf-8").read()

suggested_requirements = env_agent.get_suggestion(pip_error_log, requirements_text)
print(suggested_requirements)

python_traceback = "Traceback (most recent call last): ..."
file_path, patched_code = code_agent.get_suggestion(python_traceback)
print("Patched file:", file_path)
print(patched_code[:400], "...")

## Extending the Architecture

CodeHealer is intentionally modular. You can replace agents, customize sandbox creation, or add new healing phases. Below is a sketch of how to introduce a custom agent that adds logging around code fixes.

In [ ]:
from codehealer.agents.code_agent import CodeAgent

class AuditedCodeAgent(CodeAgent):
    """Wraps CodeAgent to persist every suggestion for review."""

    def __init__(self, repo_path, audit_log_path):
        super().__init__(repo_path)
        self.audit_log_path = audit_log_path

    def get_suggestion(self, traceback_log):
        suggestion = super().get_suggestion(traceback_log)
        if suggestion:
            file_path, new_content = suggestion
            with open(self.audit_log_path, "a", encoding="utf-8") as audit_log:
                audit_log.write(f"Patched {file_path}\n")
        return suggestion

# Swap the agent inside the healer orchestrator.
from codehealer.core.healer import Healer

healer = Healer(repo_path=target_repo_path)
healer.code_agent = AuditedCodeAgent(target_repo_path, "./audit.log")
healer.heal()

### Further Ideas

- Add a static-analysis phase before runtime execution.
- Persist structured telemetry (e.g., JSON logs) for observability.
- Swap the sandbox implementation to target remote containers or Docker Compose services.

Feel free to duplicate this notebook and adapt the examples for your own workflows!